# PDF 로딩 및 벡터 DB 저장

이력서 PDF를 로드하고, 청킹하고, 벡터 임베딩을 생성해서 PostgreSQL pgvector에 저장합니다.

In [1]:
import sys
sys.path.append('..')

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(override=True)

DATABASE_URL = os.getenv("DATABASE_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 1. PDF 로드

In [3]:
pdf_path = "../app/data/documents/이진섭_이력서.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"총 {len(documents)} 페이지 로드됨")
print(f"첫 페이지 미리보기: {documents[0].page_content[:200]}...")

총 5 페이지 로드됨
첫 페이지 미리보기: ㈜파라바라
총 3년 8개월
경기과학기술대학교
대학교(2,3년) 졸업
회사내규에 따름
직전 연봉   5,100 만원
간략 소개
[AI 기술로 서비스 문제를 해결하는 실전형 개발자]
1년간 프리랜서·공모전 경험을 거쳐 스타트업에 합류해 3년 8개월간 핵심 백엔드 개발과 팀 리딩을 담당했습니다.
현재는 엔지니어링 기반 위에 AI 기술을 결합해 단순 연동이 아닌,...


## 2. 텍스트 청킹

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
print(f"총 {len(chunks)}개의 청크 생성됨")
print(f"첫 번째 청크: {chunks[0].page_content}")

총 18개의 청크 생성됨
첫 번째 청크: ㈜파라바라
총 3년 8개월
경기과학기술대학교
대학교(2,3년) 졸업
회사내규에 따름
직전 연봉   5,100 만원
간략 소개
[AI 기술로 서비스 문제를 해결하는 실전형 개발자]
1년간 프리랜서·공모전 경험을 거쳐 스타트업에 합류해 3년 8개월간 핵심 백엔드 개발과 팀 리딩을 담당했습니다.
현재는 엔지니어링 기반 위에 AI 기술을 결합해 단순 연동이 아닌, 비용·정확도·운영을 함께 고려한 AI 시스템을 설계합니다.
[보유 스킬]
AI Agentic Engineering
  LangGraph·LangChain 기반 도메인 맞춤 Workflow / Multi-Agent / Context Engineering 설계
  임베딩 파인튜닝·멀티모달 RAG로 정확도 중심 Tool 개발
- KoBERT 분류·임베딩 병행을 통한 비용·정확도 최적화 추론 구조
AI Modeling
  YOLO·MediaPipe·LSTM 기반 실시간 멀티모달 추론 파이프라인


## 3. 임베딩 생성

In [6]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# 테스트로 첫 번째 청크만 임베딩
test_embedding = embeddings_model.embed_query(chunks[0].page_content)
print(f"임베딩 차원: {len(test_embedding)}")

/Users/seobi/PythonProjects/fastcampus_hackathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임베딩 차원: 1024


## 4. 벡터 DB에 저장

In [10]:

def remove_null_bytes(text):
    if isinstance(text, str):
        return text.replace("\x00", "")
    return text

# 청크 내의 content와 metadata 정제
for chunk in chunks:
    # 1. 본문 내용 정제
    chunk.page_content = remove_null_bytes(chunk.page_content)
    new_metadata = {}
    for key, value in chunk.metadata.items():
        new_metadata[key] = remove_null_bytes(value)
    chunk.metadata = new_metadata


In [11]:
# %%
from langchain_postgres import PGVector

# psycopg2 연결 문자열로 변환
connection_string = DATABASE_URL.replace("postgresql+asyncpg", "postgresql+psycopg2")
vector_store = PGVector.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    collection_name="my_resume",
    connection=connection_string,
    use_jsonb=True
)

print(f"총 {len(chunks)}개 청크 저장 완료!")

총 18개 청크 저장 완료!


## 5. 저장 확인

In [20]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k" :5,
        "fetch_k":10,
        "lambda_mult": 1.0
    }
)

In [22]:
results = retriever.invoke("가장 최근 ai 경험은 어떻게 돼?")
results

[Document(id='ba299b86-b718-43a8-89f3-36bec4f0dccc', metadata={'page': 3, 'source': '../app/data/documents/이진섭_이력서.pdf', 'creator': 'Chromium', 'moddate': '2026-01-27T08:43:57+00:00', 'producer': 'Skia/PDF m115', 'page_label': '4', 'total_pages': 5, 'creationdate': '2026-01-27T08:43:57+00:00'}, page_content='- [프로젝트명] 뉴스 검증 솔루션 \'클릭베이트\' (품질 좋은 뉴스 서비스)\n- [담당] 코사인 유사도을 적용하여 7일간의 기사 중복/유사도 판별 로직 구현\n- [기술] Python, FastAPI, GPT API\n원티드랩 포텐업 2기 AI Agent 최우수상 (1등)2026.01 원티드랩\nKT K intelligence>AI Agent 공모전 2025  본선 7등2025.09 KT\n정보처리기사2025.09 최종합격 한국산업인력공단\nSQL개발자(SQLD자격)2025.07 최종합격 한국데이터베이스진흥센터\nICT 콤플렉스 앱개발 챌린지 2020  108팀 중 2등(우수상)2020.12 과학기술정보통신부\n이진섭_포트폴리오.pdf포트폴리오\n작업기간2026.01.092026.01.17\n기술의 화려함보다 운영의 효율을 쫓는, 결과 중심의 실전형 AI 개발자\n[커리어 과정] "한번 맺은 인연은 끝까지, 조직과 함께 성장하는 책임감"'),
 Document(id='585514c7-a11b-4566-9515-338c668c3eb1', metadata={'page': 1, 'source': '../app/data/documents/이진섭_이력서.pdf', 'creator': 'Chromium', 'moddate': '2026-01-27T08:43:57+00:00', 'producer': 'Skia/PDF m11